In [3]:
from reproduction.mimno_embeddings import *


In [9]:
import glob
import math
import random
from helpers.embeddings.gensim_model import GensimIterator

# Corpora of files

FIXED_ORDER = sorted(glob.glob("../data/curated/corpus/pie-http/**/*.txt"))

def fixed_order():
    return []+FIXED_ORDER

def random_order():
    x = []+FIXED_ORDER
    random.shuffle(x)
    return x

def replaced_order():
    x = []+FIXED_ORDER
    random.shuffle(x)
    return sorted(x[:math.ceil(len(x)*0.66)])

# Set-up the embeddings function

In [10]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from time import time
min_count = 5

class Iterator:
    def __init__(self, files):
        self.files = files

    def iter(self):
        for fname in self.files:
            with open(fname) as fp:
                for line in fp:
                    yield line.split()
                    
    def __iter__(self):
        for fname in self.files:
            with open(fname) as fp:
                for line in fp:
                    yield line
            

def word2vec(iterator, output):
    model = Word2Vec(min_count=5, workers=4, size=100)
    t = time()
    print("Building vocab")
    model.build_vocab(iterator, progress_per=1000)
    print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
    t = time()
    print("Training")
    model.train(iterator, total_examples=model.corpus_count, epochs=50, report_delay=1)
    print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
    model.wv.save(output)
    return KeyedVectors.load(output, mmap='r')
    
model = word2vec(Iterator(FIXED_ORDER), "model.wv")

Building vocab
Time to build vocab: 0.06 mins
Training


KeyboardInterrupt: 

# Step 1 : Training Models

In [ ]:
# Set of functions that gives the right order of file
corpus_arrangements = [fixed_order, random_order, replaced_order]

# For each kind of file order
for corpus_arrangement in corpus_arrangements:
    
    # For each iteration of our algorithm
    for experience in range(EXPERIENCES_COUNT):
        
        # We generate the order
        order = corpus_arrangement()
        # We save it to a CSV to be able to source this later
        csv_writer.writerow([corpus_arrangement.__name__, str(experience)] + order)
        # Create the reader
        iterator = GensimIterator(order)
        

# Step 3 : Select random words

To have a fair representation, we should select words that appears at in different *deciles* of the corpus.

# Step 4 : Compute stability

# Step 5 : Plot stability